# IMPORTANTE

Questo file viene utilizzato per provare le chiamate API sfruttando la camera fornita dal Cisco DTLab. 

Essendo più a diretto contatto con il DTLab, alcune funzionalità, come la creazione di un broker MQTT all'interno della rete, possono essere sperimentate più facilmente

Non rappresenta quindi il prodotto finale


### Imports e credenziali ☑️

In [ ]:
import requests
import meraki
import credentials as cr
import time
import os

from pprint import pprint

API_KEY = cr.MERAKI_KEY_ABITAREIN_ADMIN
BASE_URL = "https://api.meraki.com/api/v1/organizations/"

dashboard = meraki.DashboardAPI(api_key=API_KEY, output_log=True, print_console=True, log_path='logs/') 
organizations = dashboard.organizations.getOrganizations()

### Organizzazione ☑️

In [ ]:
ORG_ID = organizations[0]['id']
print (ORG_ID)

### Network ☑️

In [ ]:
response_netID = dashboard.organizations.getOrganizationNetworks(ORG_ID)  
print(response_netID)
NETWORK_ID = response_netID[0]['id']

print(NETWORK_ID)

### Ottenimento seriali di tutti i dispositivi ☑️

In [ ]:
response_serial = dashboard.organizations.getOrganizationDevices(ORG_ID)
print(response_serial)

### Categorizzazione seriali dei dispositivi ☑️

In [ ]:


SERIAL = response_serial[0]['serial']
DEVICE_MODEL = response_serial[0]['model']
DEVICE_PRODUCT = response_serial[0]['productType']

DEVICE = {}
i = 0
# To specify a range, use slicing. For example, to get the first 3 serials:
for device in response_serial[0:3]:
    DEVICE[i] = {
        "NUMBER": i,
        "SERIAL": device['serial'],
        "DEVICE_MODEL": device['model'],
        "DEVICE_PRODUCT": device['productType'],
        "MAC_ADDRESS" : device['mac'],
        "HAS_A_UPLINK" : None,
        "DEVICE_UPLINK" : None
    }
    i = i+1
pprint(DEVICE)


### Dispositivi raggiungibili nella rete ☑️

In [ ]:
response_uplink = dashboard.organizations.getOrganizationDevicesUplinksAddressesByDevice(ORG_ID, serials=[SERIAL])

pprint(response_uplink)

### Viene effettuata una correlazione tra dispositivi raggiungibili nella rete e dispositivi presenti. Dopodiché, vengono stampati i dispositivi rilevati ☑️

In [ ]:
i = 0
UPLINK_DEVICE = {}
for device in response_uplink[0:3]:
    UPLINK_DEVICE[i] = {
        "NUMBER": i,
        "SERIAL": device['serial'],
        "DEVICE_PRODUCT": device['productType'],
        "DEVICE_UPLINK" : device['uplinks'] # Uplinks è un dizionario con due campi, indirizzi e interfacce. Indirizzi a sua volta è un dizionario con indirizzo pubblico, modalità di assegnamento, nameserver, interfaccia fisica, protocollo, indirizzo pubblico e vlan di appartenenza
    }
    i = i+1

for idx, device in DEVICE.items():
    serial = device["SERIAL"]
    uplink_info = next((uplink for uplink in UPLINK_DEVICE.values() if uplink["SERIAL"] == serial), None)
    if uplink_info:
        DEVICE[idx]["HAS_A_UPLINK"] = True
        DEVICE[idx]["DEVICE_UPLINK"] = uplink_info["DEVICE_UPLINK"]
    else:
        DEVICE[idx]["HAS_A_UPLINK"] = False
        DEVICE[idx]["DEVICE_UPLINK"] = None

pprint(DEVICE)

### Vengono individuate le camere ☑️

In [ ]:
# Trova il seriale di un dispositivo che è una camera
SERIAL_CAMERA = 'Q2LU-EDMJ-3X4W'


In [ ]:
name = 'MQTT_Broker'
host = 'mqtt.merakiabitarein.app'
port = 80


### Impostazione MQTT broker nella rete(da fare una sola volta) ☑️

In [ ]:

try:
    response = dashboard.networks.createNetworkMqttBroker(
        NETWORK_ID,
        name=name,
        host=host,
        port=port,
        security={'mode': 'none'},
        authentication={'username': 'abitarein', 'password': 'abitarein'}
    )
    print(response)
except meraki.APIError as e:
    print("Meraki API Error:", e)
    print("Response:", e.response.text)
except Exception as ex:
    print("General Exception:", ex)

### Imposta MVSense con MQTT Broker (Da fare una sola volta)

In [ ]:
response_set_mvsense = dashboard.camera.updateDeviceCameraSense(
    SERIAL_CAMERA, 
    senseEnabled=True, 
    mqttBrokerId=cr.MQTT_BROKER_ID_THU, 
    audioDetection={'enabled': False}
)

print(response_set_mvsense)



#### Codice broker mqtt

In [ ]:
import time

import paho.mqtt.client as mqtt

def start_meraki_camera_mqtt_client():
    topic = f"/testopic"
    broker = 'mqtt.merakiabitarein.app'
    port_num = 80
    username = "abitarein"
    password = "abitarein"

    def on_connect(client, userdata, flags, reason_code, properties):
        print(f"Connected with result code {reason_code}")
        # Subscribing in on_connect() means that if we lose the connection and
        # reconnect then subscriptions will be renewed.
        client.subscribe("$SYS/#")

    # The callback for when a PUBLISH message is received from the server.
    def on_message(client, userdata, msg):
        print(msg.topic+" "+str(msg.payload))

    client = mqtt.Client(callback_api_version=mqtt.CallbackAPIVersion.VERSION2, client_id="pclient")
    client.username_pw_set(username, password)
    client.on_message = on_message
    client.on_connect = on_connect

    client.connect(broker, port_num,60)
    client.subscribe(topic)

    print(f"Subscribed to topic: {topic}")
    client.loop_start()
    print("MQTT client started.")

start_meraki_camera_mqtt_client()


try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("Stopped MQTT client.")


#### publisher mqtt


In [ ]:
from random import randrange, uniform
import time

broker = "mqtt.merakiabitarein.app"
port = 80
client = mqtt.Client(callback_api_version=mqtt.CallbackAPIVersion.VERSION2, client_id="prova")
client.username_pw_set("abitarein", "abitarein")
client.connect(broker, port)

while True:
    randNumber = uniform(20.0, 21.0)
    client.publish(topic = "/testopic",  payload = randNumber)
    print("published")
    time.sleep(1)
